In [30]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd
import re

from os.path import basename

In [31]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

pd.options.display.max_colwidth = 100
print('nameFileS:'+nameFileS)
print('pathFileS:'+pathFileS)
print('rootFileS:'+rootFileS)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

0     nameFileS:parameters.json
1     nameFileS:parameters.json
2     nameFileS:parameters.json
3     nameFileS:parameters.json
4     nameFileS:parameters.json
5     nameFileS:parameters.json
6     nameFileS:parameters.json
7     nameFileS:parameters.json
8     nameFileS:parameters.json
9     nameFileS:parameters.json
10    nameFileS:parameters.json
dtype: object
0     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/l...
1     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/m...
2     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/f...
3     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/c...
4     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/b...
5     pathFileS:../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL/i..

In [32]:
dfRDC = pd.read_csv("RangeValueClassification_tot.csv", sep=';')
del dfRDC['Unnamed: 0']

In [33]:
dfRDC

,DEVICE,FUNCTION,DATA_TYPE,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,Arm,lud,int8_t,1,1,0,0,0,0,0,0
1,Arm,lud,int16_t,1,1,0,0,0,0,0,0
2,Arm,lud,int32_t,1,1,0,0,0,0,0,0
3,Arm,lud,int64_t,1,1,0,0,0,0,0,0
4,Thumb,lud,int8_t,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
214,Leon3,shell_sort,int64_t,0,0,1,3,1,1,0,0
215,RiscV,shell_sort,int8_t,0,0,1,3,1,1,0,0
216,RiscV,shell_sort,int16_t,0,0,1,3,1,1,0,0
217,RiscV,shell_sort,int32_t,0,0,1,3,1,1,0,0


In [34]:
nameFileH = [] # Affinity
pathFileH = [] # Affinity
rootFileH = [] # Affinity
nameFileM = [] # cc4csValues
pathFileM = [] # cc4csValues 
rootFileM = [] # cc4csValues
nameFileS = [] # Size SW
pathFileS = [] # Size SW
rootFileS = [] # Size SW
#nameFileC = []
#pathFileC = []
#rootFileC = []
nameFileA = [] # area HW
pathFileA = [] # area HW
rootFileA = [] # area HW

# path = os.getcwd()
for root, dirs, files in os.walk('../../../SLIDE-x-D002-Arm-Thumb-Atmega-Leon3-RiscV-Recipe/SLIDE-x-RES/RECIPE_DECIMAL'):
    for file in files:
        if file.endswith("Affinity.csv"):
            nameFileH.append(file)
            pathFileH.append(os.path.join(root, file))
            rootFileH.append(root)
        elif file.endswith("cc4csValues.csv"):
            nameFileM.append(file)
            pathFileM.append(os.path.join(root, file))
            rootFileM.append(root)
        elif file.endswith("Size.csv"):
            nameFileS.append(file)
            pathFileS.append(os.path.join(root, file))
            rootFileS.append(root)
        elif file.endswith("idAreaValuesParam.csv"):
            nameFileA.append(file)
            pathFileA.append(os.path.join(root, file))
            rootFileA.append(root)

#print('nameFileH'+str(nameFileH))
#print('pathFileH'+str(pathFileH))
#print('rootFileH'+str(rootFileH))

#print('nameFileM'+str(nameFileM))
#print('pathFileM'+str(pathFileM))
#print('rootFileM'+str(rootFileM))

#print('nameFileS'+str(nameFileH))
#print('pathFileS'+str(nameFileH))
#print('rootFileS'+str(nameFileH))

#print('nameFileA'+str(nameFileH))
#print('pathFileA'+str(nameFileH))
#print('rootFileA'+str(nameFileH))

In [35]:
# Affinity file
for i in range(0, len(pathFileH)):
    path = pathFileH[i]
    try:
        dfHTemp = pd.read_csv(path, sep=',',engine='python')
    except:
        print(path)
        
    if os.name == 'nt':
        spR = rootFileH[i].split('\\')
    else:
        spR = rootFileH[i].split('/')
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # Bambu (HW)
    board = spR[-4] # SW
    function = spR[-5]
    dfHTemp['DEVICE'] = device
    dfHTemp['BOARD'] = board
    dfHTemp['FUNCTION'] = function
    dfHTemp['DATA_TYPE'] = datatype
    dfHTemp['OPT_FLAG'] = optFlag
    dfHTemp['MV_SIZE'] = matrixVectorSize
    dfHTemp['inputName'] = dfHTemp['inputName'].map(lambda x: x.lstrip('values_'))
    dfHTemp['inputName'] = dfHTemp['inputName'].astype(int)
    dfHTemp.sort_values('inputName')
    
    if i == 0:
        dfHTot = dfHTemp
    else:
        frames = [dfHTot, dfHTemp]
        dfHTot = pd.concat(frames)
        
# a = dfHTot['Program_Volume']/3000
# a = a.values.tolist()
# dfHTot['Bugs_Delivered'] = a

In [36]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
1,19,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
2,12,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
3,38,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
4,81,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,168,34,60,18,228,52,1299.70,73649.68,0.02,...,0.0,0.466667,0.877738,0.725625,RiscV,RiscV,shell_sort,float,optO2-07,32x32
96,98,186,34,71,30,257,64,1542.00,62039.80,0.02,...,0.0,0.394366,0.848953,0.656886,RiscV,RiscV,shell_sort,float,optO2-07,32x32
97,29,192,34,75,34,267,68,1625.35,60950.72,0.03,...,0.0,0.373333,0.838743,0.633058,RiscV,RiscV,shell_sort,float,optO2-07,32x32
98,80,171,34,63,22,234,56,1358.92,66154.75,0.02,...,0.0,0.444444,0.869804,0.706511,RiscV,RiscV,shell_sort,float,optO2-07,32x32


In [37]:
# Define the function to categorize clock cycles
def categorize_opt(clockCycles, minCC, q1CC, medCC, q3CC, maxCC):
    if clockCycles == minCC:
        return 1
    elif  clockCycles > minCC and clockCycles < q1CC:
        return 2
    elif clockCycles >= q1CC and clockCycles < medCC:
        return 3
    elif clockCycles >= medCC and clockCycles < q3CC:
        return 4
    elif clockCycles >= q3CC and clockCycles < maxCC:
        return 5
    elif clockCycles == maxCC:
        return 6
    
def has_number(column):
    # Use apply to check each value and any to see if any are numeric
    return column.apply(lambda x: isinstance(x, (int, float))).any()

# cc4csValues file
for i in range(0, len(pathFileM)):
    path = pathFileM[i]
    dfMTemp = pd.read_csv(path, sep=',', quotechar='"',engine='python')
        
    if os.name == 'nt':
        spR = rootFileM[i].split('\\')
    else:
        spR = rootFileM[i].split('/')  
    
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    # board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    board = spR[-4] 
    function = spR[-5]
    dfMTemp['DEVICE'] = device
    dfMTemp['BOARD'] = board
    dfMTemp['FUNCTION'] = function
    dfMTemp['DATA_TYPE'] = datatype
    dfMTemp['OPT_FLAG'] = optFlag
    dfMTemp['MV_SIZE'] = matrixVectorSize
    dfMTemp['inputName'] = dfMTemp['id'].map(lambda x: x.lstrip('values_'))
    dfMTemp['inputName'] = dfMTemp['inputName'].astype(int)
    dfMTemp.sort_values('inputName')
    dfMTemp.drop('id', axis=1, inplace=True)
    
    if (not has_number(dfMTemp['clockCycles'])):
        dfMTemp['clockCycles'] = 0
    
    if i == 0:
        dfMTot = dfMTemp
    else:
        frames = [dfMTot, dfMTemp]
        dfMTot = pd.concat(frames)

In [38]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,10043.0,5141.0,95.8,96.2,55.6,25.59,25.58,0.01,235,685,...,15.717,22.024,Leon3,Leon3,lud,double,optO5-Ofast,32x32,11,NaN
1,12888.0,6946.0,96.9,97.2,55.6,26.95,26.94,0.01,285,835,...,16.544,19.410,Leon3,Leon3,lud,double,optO5-Ofast,32x32,19,NaN
2,5456.0,2201.0,90.2,91.2,54.8,20.17,20.15,0.02,115,325,...,18.007,12.898,Leon3,Leon3,lud,double,optO5-Ofast,32x32,12,NaN
3,5196.0,2053.0,89.6,90.7,54.8,19.76,19.74,0.02,105,295,...,18.895,14.275,Leon3,Leon3,lud,double,optO5-Ofast,32x32,38,NaN
4,7001.0,3149.0,93.0,93.7,54.8,22.49,22.48,0.01,165,475,...,15.804,7.668,Leon3,Leon3,lud,double,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,961.0,961.0,NaN,NaN,NaN,NaN,NaN,NaN,38,152,...,9.066,11.720,RiscV,RiscV,shell_sort,float,optO2-07,32x32,3,"['957209', '8845188', '126180', '1109082', '65079', '552204', '568380', '49.972', '12589408', '0..."
96,1755.0,1755.0,NaN,NaN,NaN,NaN,NaN,NaN,332,1661,...,1.628,1.885,RiscV,RiscV,shell_sort,float,optO2-07,32x32,98,"['957373', '8845376', '126239', '1109143', '65097', '552205', '568392', '49.969', '12591896', '0..."
97,2119.0,2119.0,NaN,NaN,NaN,NaN,NaN,NaN,456,2303,...,1.403,1.631,RiscV,RiscV,shell_sort,float,optO2-07,32x32,29,"['957553', '8845452', '126281', '1109164', '65103', '552204', '568397', '49.966', '12593002', '0..."
98,1112.0,1112.0,NaN,NaN,NaN,NaN,NaN,NaN,100,438,...,3.732,4.520,RiscV,RiscV,shell_sort,float,optO2-07,32x32,80,"['957197', '8845222', '126186', '1109093', '65087', '552204', '568381', '49.972', '12589906', '0..."


In [39]:
# Size SW file
for i in range(0, len(pathFileS)):
    path = pathFileS[i]
    dfSTemp = pd.read_csv(path, sep=',',engine='python')

    if os.name == 'nt':
        spR = rootFileS[i].split('\\')
    else:
        spR = rootFileS[i].split('/')   
        
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = spR[-4]
    function = spR[-5]
    dfSTemp['DEVICE'] = device
    dfSTemp['BOARD'] = board
    dfSTemp['FUNCTION'] = function
    dfSTemp['DATA_TYPE'] = datatype
    dfSTemp['OPT_FLAG'] = optFlag
    dfSTemp['MV_SIZE'] = matrixVectorSize
    dfSTemp['inputName'] = dfSTemp['inputName'].map(lambda x: x.lstrip('values_') if isinstance(x, str) else x)
    dfSTemp['inputName'] = dfSTemp['inputName'].astype(int)
    dfSTemp.sort_values('inputName')
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].map(lambda x: x.lstrip('values_'))
    #dfSTemp.iloc[:,0] = dfSTemp.iloc[:, 0].astype(int)
    #dfSTemp.sort_values(by=dfSTemp.columns[0])
    
    if i == 0:
        dfSTot = dfSTemp
    else:
        frames = [dfSTot, dfSTemp]
        dfSTot = pd.concat(frames)

In [41]:
# dfSTemp.drop(dfSTemp.index, inplace=True)
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11632.0,144.0,4648.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
1,19,11632.0,144.0,6688.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
2,12,13648.0,144.0,1384.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
3,38,13456.0,144.0,1216.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
4,81,14384.0,144.0,2464.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
95,3,5742.0,2516.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
96,98,5732.0,2556.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
97,29,5732.0,2572.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
98,80,5742.0,2524.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32


In [42]:
# area parameters file
for i in range(0, len(pathFileA)):
    path = pathFileA[i]
    dfATemp = pd.read_csv(path, sep=',',engine='python')
    
    if os.name == 'nt':
        spR = rootFileA[i].split('\\')
    else:
        spR = rootFileA[i].split('/')
    matrixVectorSize = spR[-1] 
    optFlag = spR[-2]
    datatype = spR[-3]
    device = spR[-4]
    board = 'xc7a100t-1csg324-Artix-7-200-MHz'  # spR[-4] 
    function = spR[-5]
    dfATemp['DEVICE'] = device
    dfATemp['BOARD'] = board
    dfATemp['FUNCTION'] = function
    dfATemp['DATA_TYPE'] = datatype
    dfATemp['OPT_FLAG'] = optFlag
    dfATemp['MV_SIZE'] = matrixVectorSize
    dfATemp['inputName'] = dfATemp['ID_VAL'].map(lambda x: x.lstrip('values_'))
    dfATemp['inputName'] = dfATemp['inputName'].astype(int)
    dfATemp.sort_values('ID_VAL')
    
    if i == 0:
        dfATot = dfATemp
    else:
        frames = [dfATot, dfATemp]
        dfATot = pd.concat(frames)

b = dfATot['Clock_Cycles']/dfATot['Max_Freq']
b = b.values.tolist()
dfATot['Time'] = b

c = (30/dfATot['Max_Freq'])*dfATot['Clock_Cycles']
c = c.values.tolist()
dfATot['Energy'] = c

NameError: name 'dfATot' is not defined

In [43]:
dfATot

NameError: name 'dfATot' is not defined

In [44]:
dfHTot = dfHTot.reset_index(drop=True)
dfMTot = dfMTot.reset_index(drop=True)
dfSTot = dfSTot.reset_index(drop=True)
#dfCTot = dfCTot.reset_index(drop=True)
# dfATot = dfATot.reset_index(drop=True)

In [45]:
dfHTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,BMR,DR_bit,Agpp,Aasp,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
1,19,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
2,12,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
3,38,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
4,81,658,60,405,135,1063,195,8086.59,727793.29,0.01,...,0.0,0.303704,0.800282,0.506246,Leon3,Leon3,lud,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239795,3,168,34,60,18,228,52,1299.70,73649.68,0.02,...,0.0,0.466667,0.877738,0.725625,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239796,98,186,34,71,30,257,64,1542.00,62039.80,0.02,...,0.0,0.394366,0.848953,0.656886,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239797,29,192,34,75,34,267,68,1625.35,60950.72,0.03,...,0.0,0.373333,0.838743,0.633058,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239798,80,171,34,63,22,234,56,1358.92,66154.75,0.02,...,0.0,0.444444,0.869804,0.706511,RiscV,RiscV,shell_sort,float,optO2-07,32x32


In [46]:
dfMTot

,clockCycles,assemblyInstr,cacheHit,cacheHitInstr,cacheHitData,MOPS,MIPS,MFLOPS,cInstr,cGimple,...,cc4ssa,cc4opt,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,inputName,Cache
0,10043.0,5141.0,95.8,96.2,55.6,25.59,25.58,0.01,235,685,...,15.717,22.024,Leon3,Leon3,lud,double,optO5-Ofast,32x32,11,NaN
1,12888.0,6946.0,96.9,97.2,55.6,26.95,26.94,0.01,285,835,...,16.544,19.410,Leon3,Leon3,lud,double,optO5-Ofast,32x32,19,NaN
2,5456.0,2201.0,90.2,91.2,54.8,20.17,20.15,0.02,115,325,...,18.007,12.898,Leon3,Leon3,lud,double,optO5-Ofast,32x32,12,NaN
3,5196.0,2053.0,89.6,90.7,54.8,19.76,19.74,0.02,105,295,...,18.895,14.275,Leon3,Leon3,lud,double,optO5-Ofast,32x32,38,NaN
4,7001.0,3149.0,93.0,93.7,54.8,22.49,22.48,0.01,165,475,...,15.804,7.668,Leon3,Leon3,lud,double,optO5-Ofast,32x32,81,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239795,961.0,961.0,NaN,NaN,NaN,NaN,NaN,NaN,38,152,...,9.066,11.720,RiscV,RiscV,shell_sort,float,optO2-07,32x32,3,"['957209', '8845188', '126180', '1109082', '65079', '552204', '568380', '49.972', '12589408', '0..."
239796,1755.0,1755.0,NaN,NaN,NaN,NaN,NaN,NaN,332,1661,...,1.628,1.885,RiscV,RiscV,shell_sort,float,optO2-07,32x32,98,"['957373', '8845376', '126239', '1109143', '65097', '552205', '568392', '49.969', '12591896', '0..."
239797,2119.0,2119.0,NaN,NaN,NaN,NaN,NaN,NaN,456,2303,...,1.403,1.631,RiscV,RiscV,shell_sort,float,optO2-07,32x32,29,"['957553', '8845452', '126281', '1109164', '65103', '552204', '568397', '49.966', '12593002', '0..."
239798,1112.0,1112.0,NaN,NaN,NaN,NaN,NaN,NaN,100,438,...,3.732,4.520,RiscV,RiscV,shell_sort,float,optO2-07,32x32,80,"['957197', '8845222', '126186', '1109093', '65087', '552204', '568381', '49.972', '12589906', '0..."


In [47]:
dfSTot

,inputName,text,data,bss,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE
0,11,11632.0,144.0,4648.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
1,19,11632.0,144.0,6688.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
2,12,13648.0,144.0,1384.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
3,38,13456.0,144.0,1216.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
4,81,14384.0,144.0,2464.0,Leon3,Leon3,lud,double,optO5-Ofast,32x32
...,...,...,...,...,...,...,...,...,...,...
239795,3,5742.0,2516.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239796,98,5732.0,2556.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239797,29,5732.0,2572.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32
239798,80,5742.0,2524.0,1480.0,RiscV,RiscV,shell_sort,float,optO2-07,32x32


In [48]:
dfATot

NameError: name 'dfATot' is not defined

In [49]:
# To get the common columns between the DataFrames, we can use set intersection
common_columns = set(dfHTot.columns).intersection(set(dfMTot.columns)).intersection(set(dfSTot.columns))

# Convert the intersection to a list
common_columns = list(common_columns)

# Merge the DataFrames using the common columns
dfTot = pd.merge(dfHTot, dfMTot, on=common_columns, how='outer')
dfTot = pd.merge(dfTot, dfSTot, on=common_columns, how='outer') # SW
# dfTot = pd.merge(dfTot, dfATot, on=common_columns, how='outer') # HW

# Display the resulting DataFrame
# print(dfTot)

In [50]:
dfTot

,inputName,Total_Operators,Distinct_Operators,Total_Operands,Distinct_Operands,Program_Length,Vocabulary_Size,Program_Volume,Effort,Program_Level,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,189,33,74,27,263,60,1553.51,70253.28,0.02,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
1,1,188,33,74,27,262,60,1547.61,69986.15,0.02,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
2,2,189,33,74,27,263,60,1553.51,70253.28,0.02,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
3,3,188,33,74,27,262,60,1547.61,69986.15,0.02,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
4,4,190,33,74,27,264,60,1559.42,70520.40,0.02,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248189,95,207,34,87,46,294,80,1858.65,59759.54,0.03,...,100.0,100.0,11.002,2.180,3.477,3.477,NaN,196.0,124.0,9.0
248190,96,205,34,82,41,287,75,1787.67,60780.81,0.03,...,100.0,100.0,11.280,2.274,3.589,3.589,NaN,196.0,104.0,9.0
248191,97,203,34,85,44,288,78,1810.20,59448.48,0.03,...,100.0,100.0,11.142,2.215,3.512,3.512,NaN,196.0,116.0,9.0
248192,98,176,34,65,24,241,58,1411.77,65000.40,0.02,...,100.0,100.0,12.299,2.700,3.998,3.998,NaN,196.0,36.0,9.0


In [51]:
dfTot = dfTot[ ['inputName','DEVICE','BOARD','FUNCTION','DATA_TYPE','OPT_FLAG','MV_SIZE'] + [ col for col in dfTot.columns if (col != 'inputName' and col != 'DEVICE' and col != 'BOARD' and col != 'FUNCTION' and col != 'DATA_TYPE' and col != 'OPT_FLAG' and col != 'MV_SIZE') ] ]

In [52]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,OnceEe,CallExe,cc4cs,cc4ir,cc4ssa,cc4opt,Cache,text,data,bss
0,0,Arm,Arm,bs,double,optO1-00,32x32,189,33,74,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
1,1,Arm,Arm,bs,double,optO1-00,32x32,188,33,74,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
2,2,Arm,Arm,bs,double,optO1-00,32x32,189,33,74,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
3,3,Arm,Arm,bs,double,optO1-00,32x32,188,33,74,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
4,4,Arm,Arm,bs,double,optO1-00,32x32,190,33,74,...,75.0,100.0,21.956,6.814,7.659,11.762,NaN,220.0,64.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248189,95,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,207,34,87,...,100.0,100.0,11.002,2.180,3.477,3.477,NaN,196.0,124.0,9.0
248190,96,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,205,34,82,...,100.0,100.0,11.280,2.274,3.589,3.589,NaN,196.0,104.0,9.0
248191,97,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,203,34,85,...,100.0,100.0,11.142,2.215,3.512,3.512,NaN,196.0,116.0,9.0
248192,98,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,176,34,65,...,100.0,100.0,12.299,2.700,3.998,3.998,NaN,196.0,36.0,9.0


In [53]:
dfTot = pd.merge(dfTot,dfRDC,how='left',on=['DEVICE','FUNCTION','DATA_TYPE'])

In [54]:
dfTot

,inputName,DEVICE,BOARD,FUNCTION,DATA_TYPE,OPT_FLAG,MV_SIZE,Total_Operators,Distinct_Operators,Total_Operands,...,data,bss,NUM_S,RANGE_S,NUM_SI,RANGE_SI,NUM_ARR,RANGE_ARR,NUM_MAT,RANGE_MAT
0,0,Arm,Arm,bs,double,optO1-00,32x32,189,33,74,...,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Arm,Arm,bs,double,optO1-00,32x32,188,33,74,...,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Arm,Arm,bs,double,optO1-00,32x32,189,33,74,...,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Arm,Arm,bs,double,optO1-00,32x32,188,33,74,...,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Arm,Arm,bs,double,optO1-00,32x32,190,33,74,...,64.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248189,95,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,207,34,87,...,124.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248190,96,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,205,34,82,...,104.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248191,97,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,203,34,85,...,116.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248192,98,Thumb,Thumb,shell_sort,float,optO6-Og,32x32,176,34,65,...,36.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
dfTot.rename(columns={'NUM_S':'SCALAR_INPUT',
                      'RANGE_S':'RANGE_SCALAR_VALUES',
                      'NUM_SI':'SCALAR_INDEX_INPUT',
                      'RANGE_SI':'RANGE_SCALAR_INDEX_VALUES',
                      'NUM_ARR':'ARRAY_INPUT',
                      'RANGE_ARR':'RANGE_ARRAY_INPUT',
                      'NUM_MAT':'MAT_INPUT',
                      'RANGE_MAT':'RANGE_MAT_INPUT'}, inplace=True)

In [56]:
dfTot.to_csv("TotalParameterMatrix.csv", sep=';', index=False, encoding='utf-8')

In [6]:
import pandas as pd
import os

def split_csv_by_column(input_file, column_name):
    # Read the CSV file
    df = pd.read_csv(input_file, sep=';')

    # Get unique values in the specified column
    unique_values = df[column_name].unique()

    # Get the directory of the input file
    input_dir = os.path.dirname(input_file)

    # For each unique value, create a subset of the DataFrame and save it as a CSV file
    for value in unique_values:
        subset_df = df[df[column_name] == value]
        output_file = os.path.join(input_dir, f"{value}-RECIPE-DECIMAL.csv")
        subset_df.to_csv(output_file, index=False)
        print(f"Created file: {output_file}")

input_file = 'Thumb-Arm-RiscV-Leon3-Atmega-RECIPE-DECIMAL.csv'  # replace with the path to your CSV file
column_name = 'DEVICE'  # replace with the name of the column to split by

split_csv_by_column(input_file, column_name)

/tmp/ipykernel_5846/1873799936.py:6: DtypeWarning: Columns (131) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file, sep=';')


Created file: Arm-RECIPE-DECIMAL.csv
Created file: Atmega328p-RECIPE-DECIMAL.csv
Created file: Leon3-RECIPE-DECIMAL.csv
Created file: RiscV-RECIPE-DECIMAL.csv
Created file: Thumb-RECIPE-DECIMAL.csv


In [21]:
import pandas as pd
import os

def merge_csv_files(file1, file2):
    # Read the CSV files
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Concatenate the DataFrames, axis=1 to concatenate columns
    merged_df = pd.concat([df1, df2], axis=1)

    # Get the directory of the input files
    input_dir = os.path.dirname(file1)

    # Define the output file path
    output_file = os.path.join(input_dir, 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv')

    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as: {output_file}")

# Example usage
file1 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-1.csv'  # replace with the path to your first CSV file
file2 = 'xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL-2.csv'  # replace with the path to your second CSV file

merge_csv_files(file1, file2)

Merged file saved as: xc7a100t-1csg324-Artix-7-200-MHz-RECIPE-DECIMAL.csv
